### IESB
___
### Especialização em Inteligência Artificial
#### Disciplina: Fundamentos de Inteligência Artificial
#### Docente: Tatiana Tavares
#### Discente: Henrique Brandão

Proposta:

Utilizando a toolbox de sua preferência, realize o treinamento de uma  rede  neural  artificial  do  tipo  Perceptron  Multicamadas  para resolver  um  problema  de classificação multiclasses. Descreva os hiper-parâmetros do seu modelo (número de neurônios,  número de camadas, taxa de aprendizagem, função de ativação, entre outros) e discuta aspectos de desempenho (porcentagem de classificaçãocorreta).

In [1]:
import numpy as np
import pandas as pd

from random import choice

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.backend import clear_session

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

Primeiramente, vamos carregar nossos dados para a memória:

In [2]:
# Dados podem ser obtidos em: https://archive.ics.uci.edu/ml/datasets/iris
df = pd.read_csv(filepath_or_buffer='iris.csv')
df.shape

(150, 5)

In [3]:
df.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Nosso *dataset* tem 5 colunas, quatro (*sepal length*, *sepal width*, *petal length*, *petal width*) representam características relativas ao nossos registros enquanto a coluna *class* nos informa a categoria (classe) do registro.

Vamos separar os dados de entrada (`features`) e a classe (`target`) em dois conjuntos: treinamento e validação (teste) para começar a construir e testar nossos modelos:

In [4]:
x = df.drop(labels=['class'], axis=1)
y = df['class']
x.shape, y.shape

((150, 4), (150,))

Como nosso problema consiste em uma classificação com várias possibilidades, vamos transformar nosso dado categórico para um formato vetorial, onde a **posição** do valor `1` no vetor significará o **valor real** da nossa classe:

In [5]:
label_encoder = LabelEncoder()

In [6]:
_ = label_encoder.fit_transform(y)

In [7]:
y_vec = np_utils.to_categorical(_)
y_vec.shape

(150, 3)

In [8]:
label_encoder.classes_

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [9]:
x.iloc[0], y_vec[0]

(sepal length    5.1
 sepal width     3.5
 petal length    1.4
 petal width     0.2
 Name: 0, dtype: float64,
 array([1., 0., 0.], dtype=float32))

Nossa classe agora está em um formato vetorial, para retorná-la ao seu formato original e ver que o processo ocorreu de forma fidedigna, podemos fazer o seguinte:

In [10]:
indice = choice(range(x.shape[0]))
_ = df['class'].iloc[indice] == label_encoder.classes_[np.argmax(y_vec[indice])]
print(f'Verificando o índice aleatório #{indice}: {_}')

Verificando o índice aleatório #129: True


In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y_vec, test_size=0.25, random_state=7)

f'xtrain: {xtrain.shape}, ytrain: {ytrain.shape}, xtest: {xtest.shape}, ytest: {ytest.shape}'

'xtrain: (112, 4), ytrain: (112, 3), xtest: (38, 4), ytest: (38, 3)'

# Rede Neural

Vamos criar uma rede neural simples e avaliar seu desempenho nos nossos dados:

In [12]:
def rede_sequencial(units):
    clear_session()
    clf = Sequential()
    # entrada
    clf.add(Dense(units=units, activation='relu', input_dim=4))
    # oculta #1
    clf.add(Dense(units=units, activation='relu'))
    # oculta #2
    clf.add(Dense(units=units, activation='relu'))
    # saida
    clf.add(Dense(units=3, activation='softmax'))
    clf.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print(clf.summary())
    return clf

Realizando o treino da rede:

In [13]:
nn = rede_sequencial(units=8)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 40        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 27        
Total params: 211
Trainable params: 211
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
nn.fit(x=xtrain, y=ytrain, epochs=1400, shuffle=True)

Epoch 1/1400
4/4 [==============================] - 7s 1ms/step - loss: 4.5649 - categorical_accuracy: 0.3351
Epoch 2/1400
4/4 [==============================] - 0s 1ms/step - loss: 4.1329 - categorical_accuracy: 0.3268
Epoch 3/1400
4/4 [==============================] - 0s 1ms/step - loss: 3.7160 - categorical_accuracy: 0.3674
Epoch 4/1400
4/4 [==============================] - 0s 2ms/step - loss: 3.6143 - categorical_accuracy: 0.3580
Epoch 5/1400
4/4 [==============================] - 0s 1ms/step - loss: 3.2846 - categorical_accuracy: 0.3320
Epoch 6/1400
4/4 [==============================] - 0s 1ms/step - loss: 3.1763 - categorical_accuracy: 0.3372
Epoch 7/1400
4/4 [==============================] - 0s 830us/step - loss: 2.9594 - categorical_accuracy: 0.3362
Epoch 8/1400
4/4 [==============================] - 0s 1ms/step - loss: 2.6434 - categorical_accuracy: 0.3476
Epoch 9/1400
4/4 [==============================] - 0s 2ms/step - loss: 2.7329 - categorical_accuracy: 0.3112
Epoch 10

4/4 [==============================] - 0s 802us/step - loss: 0.9731 - categorical_accuracy: 0.4188
Epoch 75/1400
4/4 [==============================] - 0s 822us/step - loss: 0.9578 - categorical_accuracy: 0.4321
Epoch 76/1400
4/4 [==============================] - 0s 1ms/step - loss: 0.9506 - categorical_accuracy: 0.4820
Epoch 77/1400
4/4 [==============================] - 0s 1ms/step - loss: 0.9541 - categorical_accuracy: 0.4827
Epoch 78/1400
4/4 [==============================] - 0s 848us/step - loss: 0.9468 - categorical_accuracy: 0.4905
Epoch 79/1400
4/4 [==============================] - 0s 909us/step - loss: 0.9417 - categorical_accuracy: 0.5262
Epoch 80/1400
4/4 [==============================] - 0s 894us/step - loss: 0.9273 - categorical_accuracy: 0.5805
Epoch 81/1400
4/4 [==============================] - 0s 841us/step - loss: 0.9092 - categorical_accuracy: 0.6369
Epoch 82/1400
4/4 [==============================] - 0s 1ms/step - loss: 0.8990 - categorical_accuracy: 0.6991
Epo

4/4 [==============================] - 0s 810us/step - loss: 0.4567 - categorical_accuracy: 0.9417
Epoch 147/1400
4/4 [==============================] - 0s 878us/step - loss: 0.4612 - categorical_accuracy: 0.9396
Epoch 148/1400
4/4 [==============================] - 0s 838us/step - loss: 0.4682 - categorical_accuracy: 0.9365
Epoch 149/1400
4/4 [==============================] - 0s 723us/step - loss: 0.4599 - categorical_accuracy: 0.9260
Epoch 150/1400
4/4 [==============================] - 0s 757us/step - loss: 0.4468 - categorical_accuracy: 0.9260
Epoch 151/1400
4/4 [==============================] - 0s 920us/step - loss: 0.4571 - categorical_accuracy: 0.9125
Epoch 152/1400
4/4 [==============================] - 0s 769us/step - loss: 0.4502 - categorical_accuracy: 0.9260
Epoch 153/1400
4/4 [==============================] - 0s 1ms/step - loss: 0.4164 - categorical_accuracy: 0.9437
Epoch 154/1400
4/4 [==============================] - 0s 1ms/step - loss: 0.4541 - categorical_accuracy: 

4/4 [==============================] - 0s 736us/step - loss: 0.1916 - categorical_accuracy: 0.9722
Epoch 219/1400
4/4 [==============================] - 0s 765us/step - loss: 0.2002 - categorical_accuracy: 0.9555
Epoch 220/1400
4/4 [==============================] - 0s 906us/step - loss: 0.1833 - categorical_accuracy: 0.9722
Epoch 221/1400
4/4 [==============================] - 0s 851us/step - loss: 0.1807 - categorical_accuracy: 0.9670
Epoch 222/1400
4/4 [==============================] - 0s 816us/step - loss: 0.2017 - categorical_accuracy: 0.9586
Epoch 223/1400
4/4 [==============================] - 0s 757us/step - loss: 0.1841 - categorical_accuracy: 0.9799
Epoch 224/1400
4/4 [==============================] - 0s 816us/step - loss: 0.1935 - categorical_accuracy: 0.9705
Epoch 225/1400
4/4 [==============================] - 0s 839us/step - loss: 0.1820 - categorical_accuracy: 0.9789
Epoch 226/1400
4/4 [==============================] - 0s 888us/step - loss: 0.1766 - categorical_accura

4/4 [==============================] - 0s 958us/step - loss: 0.1046 - categorical_accuracy: 0.9690
Epoch 291/1400
4/4 [==============================] - 0s 738us/step - loss: 0.1193 - categorical_accuracy: 0.9513
Epoch 292/1400
4/4 [==============================] - 0s 807us/step - loss: 0.1094 - categorical_accuracy: 0.9576
Epoch 293/1400
4/4 [==============================] - 0s 776us/step - loss: 0.1067 - categorical_accuracy: 0.9586
Epoch 294/1400
4/4 [==============================] - 0s 839us/step - loss: 0.0877 - categorical_accuracy: 0.9836
Epoch 295/1400
4/4 [==============================] - 0s 705us/step - loss: 0.0957 - categorical_accuracy: 0.9670
Epoch 296/1400
4/4 [==============================] - 0s 927us/step - loss: 0.0968 - categorical_accuracy: 0.9815
Epoch 297/1400
4/4 [==============================] - 0s 1ms/step - loss: 0.1009 - categorical_accuracy: 0.9722
Epoch 298/1400
4/4 [==============================] - 0s 871us/step - loss: 0.1011 - categorical_accuracy

...

4/4 [==============================] - 0s 805us/step - loss: 0.0393 - categorical_accuracy: 0.9856
Epoch 1079/1400
4/4 [==============================] - 0s 951us/step - loss: 0.0285 - categorical_accuracy: 0.9887
Epoch 1080/1400
4/4 [==============================] - 0s 720us/step - loss: 0.0515 - categorical_accuracy: 0.9762
Epoch 1081/1400
4/4 [==============================] - 0s 852us/step - loss: 0.0297 - categorical_accuracy: 0.9876
Epoch 1082/1400
4/4 [==============================] - 0s 704us/step - loss: 0.0408 - categorical_accuracy: 0.9824
Epoch 1083/1400
4/4 [==============================] - 0s 1ms/step - loss: 0.0372 - categorical_accuracy: 0.9856
Epoch 1084/1400
4/4 [==============================] - 0s 754us/step - loss: 0.0481 - categorical_accuracy: 0.9762
Epoch 1085/1400
4/4 [==============================] - 0s 669us/step - loss: 0.0490 - categorical_accuracy: 0.9762
Epoch 1086/1400
4/4 [==============================] - 0s 833us/step - loss: 0.0602 - categorical_

4/4 [==============================] - 0s 3ms/step - loss: 0.0310 - categorical_accuracy: 0.9876
Epoch 1150/1400
4/4 [==============================] - 0s 1ms/step - loss: 0.0457 - categorical_accuracy: 0.9793
Epoch 1151/1400
4/4 [==============================] - 0s 815us/step - loss: 0.0384 - categorical_accuracy: 0.9856
Epoch 1152/1400
4/4 [==============================] - 0s 880us/step - loss: 0.0394 - categorical_accuracy: 0.9824
Epoch 1153/1400
4/4 [==============================] - 0s 798us/step - loss: 0.0490 - categorical_accuracy: 0.9793
Epoch 1154/1400
4/4 [==============================] - 0s 896us/step - loss: 0.0253 - categorical_accuracy: 0.9929
Epoch 1155/1400
4/4 [==============================] - 0s 911us/step - loss: 0.0470 - categorical_accuracy: 0.9793
Epoch 1156/1400
4/4 [==============================] - 0s 773us/step - loss: 0.0252 - categorical_accuracy: 0.9908
Epoch 1157/1400
4/4 [==============================] - 0s 1ms/step - loss: 0.0324 - categorical_accu

4/4 [==============================] - 0s 804us/step - loss: 0.0282 - categorical_accuracy: 0.9887
Epoch 1221/1400
4/4 [==============================] - 0s 748us/step - loss: 0.0321 - categorical_accuracy: 0.9876
Epoch 1222/1400
4/4 [==============================] - 0s 939us/step - loss: 0.0407 - categorical_accuracy: 0.9824
Epoch 1223/1400
4/4 [==============================] - 0s 903us/step - loss: 0.0519 - categorical_accuracy: 0.9762
Epoch 1224/1400
4/4 [==============================] - 0s 878us/step - loss: 0.0285 - categorical_accuracy: 0.9887
Epoch 1225/1400
4/4 [==============================] - 0s 860us/step - loss: 0.0330 - categorical_accuracy: 0.9876
Epoch 1226/1400
4/4 [==============================] - 0s 851us/step - loss: 0.0354 - categorical_accuracy: 0.9856
Epoch 1227/1400
4/4 [==============================] - 0s 752us/step - loss: 0.0511 - categorical_accuracy: 0.9762
Epoch 1228/1400
4/4 [==============================] - 0s 888us/step - loss: 0.0514 - categorica

4/4 [==============================] - 0s 757us/step - loss: 0.0383 - categorical_accuracy: 0.9824
Epoch 1292/1400
4/4 [==============================] - 0s 797us/step - loss: 0.0376 - categorical_accuracy: 0.9814
Epoch 1293/1400
4/4 [==============================] - 0s 776us/step - loss: 0.0273 - categorical_accuracy: 0.9887
Epoch 1294/1400
4/4 [==============================] - 0s 749us/step - loss: 0.0472 - categorical_accuracy: 0.9762
Epoch 1295/1400
4/4 [==============================] - 0s 794us/step - loss: 0.0331 - categorical_accuracy: 0.9876
Epoch 1296/1400
4/4 [==============================] - 0s 770us/step - loss: 0.0455 - categorical_accuracy: 0.9814
Epoch 1297/1400
4/4 [==============================] - 0s 840us/step - loss: 0.0421 - categorical_accuracy: 0.9814
Epoch 1298/1400
4/4 [==============================] - 0s 848us/step - loss: 0.0466 - categorical_accuracy: 0.9762
Epoch 1299/1400
4/4 [==============================] - 0s 785us/step - loss: 0.0274 - categorica

4/4 [==============================] - 0s 1ms/step - loss: 0.0249 - categorical_accuracy: 0.9908
Epoch 1363/1400
4/4 [==============================] - 0s 841us/step - loss: 0.0386 - categorical_accuracy: 0.9824
Epoch 1364/1400
4/4 [==============================] - 0s 834us/step - loss: 0.0344 - categorical_accuracy: 0.9876
Epoch 1365/1400
4/4 [==============================] - 0s 754us/step - loss: 0.0413 - categorical_accuracy: 0.9793
Epoch 1366/1400
4/4 [==============================] - 0s 881us/step - loss: 0.0262 - categorical_accuracy: 0.9876
Epoch 1367/1400
4/4 [==============================] - 0s 914us/step - loss: 0.0209 - categorical_accuracy: 0.9929
Epoch 1368/1400
4/4 [==============================] - 0s 1ms/step - loss: 0.0609 - categorical_accuracy: 0.9699
Epoch 1369/1400
4/4 [==============================] - 0s 943us/step - loss: 0.0314 - categorical_accuracy: 0.9876
Epoch 1370/1400
4/4 [==============================] - 0s 845us/step - loss: 0.0484 - categorical_ac

Testando o desempenho na segunda parte dos dados:

In [15]:
pred = nn.predict(x=xtest)
pred[:5]

array([[8.3312619e-09, 1.6517485e-02, 9.8348248e-01],
       [5.1892589e-06, 7.6877880e-01, 2.3121603e-01],
       [9.9986649e-01, 1.3354323e-04, 1.8363607e-12],
       [5.8906767e-06, 8.8039225e-01, 1.1960184e-01],
       [1.3812244e-08, 6.6447565e-03, 9.9335527e-01]], dtype=float32)

Vamos converter os dados de saída da predição para um formato um formato análogo a nossa classe vetorizada:

In [16]:
pred_vec = [np.argmax(x) for x in pred]
pred_vec[:5]

[2, 1, 0, 1, 2]

In [17]:
ytest_vec = [np.argmax(x) for x in ytest]
ytest_vec[:5]

[2, 1, 0, 1, 2]

In [18]:
print(f'# Matriz de confusão:\n{confusion_matrix(y_true=ytest_vec, y_pred=ytest_vec)}')

# Matriz de confusão:
[[11  0  0]
 [ 0 14  0]
 [ 0  0 13]]


In [19]:
print(f'# Relatório de classificação:\n{classification_report(y_true=ytest_vec, y_pred=ytest_vec)}')

# Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00        13

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



Podemos observar que todas as métricas do nosso modelo alcançaram 100% de aproveitamento!

- Contextualizar sua classificação, qual o problema a ser solucionado com a sua classificação.
        Dadas as medidas relativas às pétalas e sépalas das flores, seria possível classificá-las através de um algoritmo?
        R: Sim. Podemos observar o bom desempenho desse modelo.

- Levantar as dificuldades, limitações, inconsistências e imprecisões nas soluções propostas.
        Poucos dados no dataset.

- Sugerir futuras implementações.
        Testar o modelo em mais registros.

- Para o seu banco de dados existem outros trabalhos de classificação na literatura? Compare-os.
        Sim, bastante. Esse dataset é comumente utilizado para iniciar o processo de aprendizado de redes neurais.